# MILESTONE 2

In order to get a feel for what we're trying to achieve in our project, you'll find below a descriptive analysis of 3 specificaly curated subsets of the Reddit dataset.
These datasets consist of the comments posted on the famous subreddit __r/politics__ during the months of October 2008, 2012 and 2016. The United States presidential elections always happen at the beggining of November, which is why the datasets timeframes were picked that way. We aim to uncover possible relationships between the outcomes of the elections and influential comments or users.  
  
These 3 datasets were obtained using the python script *script.py* the following way (on MacOS).  
1. Upload the files script.sh and script.py using the following command:  
```bash 
scp -r -p /path/to/script.py GASPAR@iccluster028.iccluster.epfl.ch:/home/GASPAR/script.py  
scp -r -p /path/to/script.sh GASPAR@iccluster028.iccluster.epfl.ch:/home/GASPAR/script.sh
```
2. Connect to the cluster using a second terminal and run the scripts:  
```bash 
ssh GASPAR@iccluster028.iccluster.epfl.ch  
sh script.sh script.py  
```
3. Store the output in convenient JSON files in the home folder:  
```bash 
hadoop fs -cat /user/GASPAR/sample_2008_10.json/* > sample_2008_10.json  
hadoop fs -cat /user/GASPAR/sample_2012_10.json/* > sample_2012_10.json  
hadoop fs -cat /user/GASPAR/sample_2016_10.json/* > sample_2016_10.json  
```
4. Download the JSON files locally:  
```bash 
scp GASPAR@iccluster028.iccluster.epfl.ch:/home/GASPAR/sample_2008_10.json /local/path/to/sample_2008_10.json  
scp GASPAR@iccluster028.iccluster.epfl.ch:/home/GASPAR/sample_2012_10.json /local/path/to/sample_2012_10.json  
scp GASPAR@iccluster028.iccluster.epfl.ch:/home/GASPAR/sample_2016_10.json /local/path/to/sample_2016_10.json
```
  
The scripts already take care of removing unnecessary columns, and only keep useful records i.e. records whose body wasn't deleted.

In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_PATH = "data/"

In [3]:
df_2008 = pd.read_json(DATA_PATH + "sample_2008_10.json", lines=True)
df_2008.head()

,author,body,created_utc,gilded,score
0,Facepuncher,dun dun dunnnnnnn!,1222819202,0,3
1,Mr_Smartypants,"Yeah, but he put his hand back up when the guy...",1222819205,0,15
2,kwan,There are more than two candidates.,1222819215,0,0
3,stone11,The 'script' being reality.,1222819232,0,3
4,digiphaze,I still haven't heard a good argument as to WH...,1222819276,0,5


In [4]:
df_2012 = pd.read_json(DATA_PATH + "sample_2012_10.json", lines=True)
df_2012.head()

,author,body,created_utc,gilded,score
0,Offensive_Brute,my breed will never die out. when your kind is...,1349049610,0,-5
1,Averyphotog,"To be fair, it just might be too complicated t...",1349049611,0,2
2,seven_seven,Why is there evidence you can cure other menta...,1349049618,0,-4
3,Dinosquid,this title is awful.,1349049629,0,4
4,NickRausch,Unless it is the Black Panthers!,1349049634,0,-2


In [5]:
df_2016 = pd.read_json(DATA_PATH + "sample_2016_10.json", lines=True)
df_2016.head()

,author,body,created_utc,gilded,score
0,creepyeyes,&gt;Dollar coins finally catch on thanks to tr...,1475280000,0,2
1,Ninbyo,It's a tracking poll. Which means they track t...,1475280002,0,3
2,ArsenyKz,"So, do you think that the Civil Rights Act (or...",1475280008,0,1
3,BasketcaseDeplorable,I personally don't see anything wrong here. Th...,1475280010,0,5
4,ChanceTheDog,I admit I liked Bernie and I think Trump V Ber...,1475280012,0,1
